# HW2 - Mikołaj Gałkowski

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
data = pd.read_csv('preparedData.csv')

In [10]:
data.shape

(98052, 169)

In [11]:
data.head(5)

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,A1Cresult,...,glyburide-metformin_Down,glyburide-metformin_No,glyburide-metformin_Steady,glyburide-metformin_Up,glipizide-metformin_No,glipizide-metformin_Steady,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes
0,0.153846,0.442748,0.000000,0.2125,0.000000,0.0,0.0,0.461538,0.0,0.0,...,0,1,0,0,1,0,1,0,0,1
1,0.076923,0.076336,0.833333,0.1500,1.414214,0.0,1.0,0.230769,0.0,0.0,...,0,1,0,0,1,0,0,1,0,1
2,0.076923,0.328244,0.166667,0.1875,0.000000,0.0,0.0,0.307692,0.0,0.0,...,0,1,0,0,1,0,1,0,0,1
3,0.000000,0.381679,0.000000,0.0875,0.000000,0.0,0.0,0.153846,0.0,0.0,...,0,1,0,0,1,0,1,0,0,1
4,0.153846,0.229008,1.000000,0.1875,0.000000,0.0,0.0,0.461538,0.0,0.0,...,0,1,0,0,1,0,0,1,0,1


In [12]:
y = data[['readmitted']]
data.drop('readmitted', inplace = True, axis = 1)
X = data 

## Wybór modelu - XGBoostClassifier

Podział na zbiór treningowy i testowy.

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Model XGBClassifier z domyślnymi parametrami

In [18]:
import xgboost as xgb

In [34]:
classifier = xgb.XGBClassifier()

classifier.fit(X_train, y_train.values.ravel())

C:\Users\mikol\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:52:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [35]:
pred_class = classifier.predict(X_test)
pred_proba = classifier.predict_proba(X_test)

In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

In [95]:
print("Accuracy is {0:.4f}".format(accuracy_score(y_test, pred_class)))
print("Precision is {0:.4f}".format(precision_score(y_test, pred_class)))
print("Recall is {0:.4f}".format(recall_score(y_test, pred_class)))
print("AUC score is {0:.4f}".format(roc_auc_score(y_test, pred_proba[:,-1])))

Accuracy is 0.6370
Precision is 0.6206
Recall is 0.5520
AUC score is 0.6913


## Ręczne poprawienie hiperparemetrów

In [90]:
classifier2 = xgb.XGBClassifier(max_depth = 15, learning_rate = 0.05, gamma = 3, n_estimators = 1000, 
                                reg_alpha = 1.5, reg_lambda = 3)

classifier2.fit(X_train, y_train.values.ravel())

C:\Users\mikol\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:03:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=3, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.05, max_delta_step=0,
              max_depth=15, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=1.5, reg_lambda=3, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [96]:
pred_class2 = classifier2.predict(X_test)
pred_proba2 = classifier2.predict_proba(X_test)

In [97]:
print("Accuracy is {0:.4f}".format(accuracy_score(y_test, pred_class2)))
print("Precision is {0:.4f}".format(precision_score(y_test, pred_class2)))
print("Recall is {0:.4f}".format(recall_score(y_test, pred_class2)))
print("AUC score is {0:.4f}".format(roc_auc_score(y_test, pred_proba2[:,-1])))

Accuracy is 0.6360
Precision is 0.6160
Recall is 0.5638
AUC score is 0.6911


Otrzymany wynik AUC score jest niestety trochę gorszy.

## GridSearch

In [38]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

Parametry, które wydają się warte zooptymalizowania to `gamma`, `learning_rate`, `max_depth`, `n_estimators`, `reg_alpha` i `reg_lambda`.

In [74]:
xgbc0 = xgb.XGBClassifier(objective='binary:logistic',
                          booster='gbtree',
                          eval_metric='auc',
                          tree_method='hist',
                          grow_policy='lossguide',
                          use_label_encoder=False)

param_grid = {'gamma': [0,1,5,10],
              'learning_rate': [0.01,0.1],
              'max_depth': [5,10,15],
              'n_estimators': [50,100,150],
              'reg_alpha': [0,1,3],
              'reg_lambda': [0,1,3]}

clf = GridSearchCV(xgbc0, param_grid = param_grid, scoring = "roc_auc", return_train_score=True, verbose = 10, cv = 3)
clf.fit(X_train, y_train.values.ravel())

Fitting 3 folds for each of 648 candidates, totalling 1944 fits
[CV 1/3; 1/648] START gamma=0, learning_rate=0.01, max_depth=5, n_estimators=50, reg_alpha=0, reg_lambda=0
[CV 1/3; 1/648] END gamma=0, learning_rate=0.01, max_depth=5, n_estimators=50, reg_alpha=0, reg_lambda=0;, score=(train=0.670, test=0.670) total time=   0.6s
[CV 2/3; 1/648] START gamma=0, learning_rate=0.01, max_depth=5, n_estimators=50, reg_alpha=0, reg_lambda=0
[CV 2/3; 1/648] END gamma=0, learning_rate=0.01, max_depth=5, n_estimators=50, reg_alpha=0, reg_lambda=0;, score=(train=0.674, test=0.668) total time=   0.7s
[CV 3/3; 1/648] START gamma=0, learning_rate=0.01, max_depth=5, n_estimators=50, reg_alpha=0, reg_lambda=0
[CV 3/3; 1/648] END gamma=0, learning_rate=0.01, max_depth=5, n_estimators=50, reg_alpha=0, reg_lambda=0;, score=(train=0.673, test=0.666) total time=   0.6s
[CV 1/3; 2/648] START gamma=0, learning_rate=0.01, max_depth=5, n_estimators=50, reg_alpha=0, reg_lambda=1
[CV 1/3; 2/648] END gamma=0, learn

[CV 2/3; 11/648] END gamma=0, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0, reg_lambda=1;, score=(train=0.679, test=0.672) total time=   0.9s
[CV 3/3; 11/648] START gamma=0, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0, reg_lambda=1
[CV 3/3; 11/648] END gamma=0, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0, reg_lambda=1;, score=(train=0.679, test=0.671) total time=   0.9s
[CV 1/3; 12/648] START gamma=0, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0, reg_lambda=3
[CV 1/3; 12/648] END gamma=0, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0, reg_lambda=3;, score=(train=0.675, test=0.675) total time=   0.9s
[CV 2/3; 12/648] START gamma=0, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0, reg_lambda=3
[CV 2/3; 12/648] END gamma=0, learning_rate=0.01, max_depth=5, n_estimators=100, reg_alpha=0, reg_lambda=3;, score=(train=0.678, test=0.672) total time=   1.0s
[CV 3/3; 12/648] START gamma=0, l

[CV 3/3; 21/648] END gamma=0, learning_rate=0.01, max_depth=5, n_estimators=150, reg_alpha=0, reg_lambda=3;, score=(train=0.682, test=0.674) total time=   1.6s
[CV 1/3; 22/648] START gamma=0, learning_rate=0.01, max_depth=5, n_estimators=150, reg_alpha=1, reg_lambda=0
[CV 1/3; 22/648] END gamma=0, learning_rate=0.01, max_depth=5, n_estimators=150, reg_alpha=1, reg_lambda=0;, score=(train=0.679, test=0.679) total time=   1.4s
[CV 2/3; 22/648] START gamma=0, learning_rate=0.01, max_depth=5, n_estimators=150, reg_alpha=1, reg_lambda=0
[CV 2/3; 22/648] END gamma=0, learning_rate=0.01, max_depth=5, n_estimators=150, reg_alpha=1, reg_lambda=0;, score=(train=0.682, test=0.675) total time=   1.4s
[CV 3/3; 22/648] START gamma=0, learning_rate=0.01, max_depth=5, n_estimators=150, reg_alpha=1, reg_lambda=0
[CV 3/3; 22/648] END gamma=0, learning_rate=0.01, max_depth=5, n_estimators=150, reg_alpha=1, reg_lambda=0;, score=(train=0.682, test=0.674) total time=   1.3s
[CV 1/3; 23/648] START gamma=0, l

[CV 1/3; 32/648] END gamma=0, learning_rate=0.01, max_depth=10, n_estimators=50, reg_alpha=1, reg_lambda=1;, score=(train=0.722, test=0.682) total time=   1.6s
[CV 2/3; 32/648] START gamma=0, learning_rate=0.01, max_depth=10, n_estimators=50, reg_alpha=1, reg_lambda=1
[CV 2/3; 32/648] END gamma=0, learning_rate=0.01, max_depth=10, n_estimators=50, reg_alpha=1, reg_lambda=1;, score=(train=0.728, test=0.677) total time=   1.6s
[CV 3/3; 32/648] START gamma=0, learning_rate=0.01, max_depth=10, n_estimators=50, reg_alpha=1, reg_lambda=1
[CV 3/3; 32/648] END gamma=0, learning_rate=0.01, max_depth=10, n_estimators=50, reg_alpha=1, reg_lambda=1;, score=(train=0.725, test=0.674) total time=   1.6s
[CV 1/3; 33/648] START gamma=0, learning_rate=0.01, max_depth=10, n_estimators=50, reg_alpha=1, reg_lambda=3
[CV 1/3; 33/648] END gamma=0, learning_rate=0.01, max_depth=10, n_estimators=50, reg_alpha=1, reg_lambda=3;, score=(train=0.720, test=0.683) total time=   1.4s
[CV 2/3; 33/648] START gamma=0, l

[CV 2/3; 42/648] END gamma=0, learning_rate=0.01, max_depth=10, n_estimators=100, reg_alpha=1, reg_lambda=3;, score=(train=0.733, test=0.683) total time=   2.5s
[CV 3/3; 42/648] START gamma=0, learning_rate=0.01, max_depth=10, n_estimators=100, reg_alpha=1, reg_lambda=3
[CV 3/3; 42/648] END gamma=0, learning_rate=0.01, max_depth=10, n_estimators=100, reg_alpha=1, reg_lambda=3;, score=(train=0.729, test=0.678) total time=   3.2s
[CV 1/3; 43/648] START gamma=0, learning_rate=0.01, max_depth=10, n_estimators=100, reg_alpha=3, reg_lambda=0
[CV 1/3; 43/648] END gamma=0, learning_rate=0.01, max_depth=10, n_estimators=100, reg_alpha=3, reg_lambda=0;, score=(train=0.725, test=0.686) total time=   2.8s
[CV 2/3; 43/648] START gamma=0, learning_rate=0.01, max_depth=10, n_estimators=100, reg_alpha=3, reg_lambda=0
[CV 2/3; 43/648] END gamma=0, learning_rate=0.01, max_depth=10, n_estimators=100, reg_alpha=3, reg_lambda=0;, score=(train=0.731, test=0.683) total time=   2.9s
[CV 3/3; 43/648] START gam

...

[CV 1/3; 609/648] END gamma=10, learning_rate=0.1, max_depth=10, n_estimators=100, reg_alpha=1, reg_lambda=3;, score=(train=0.708, test=0.696) total time=   1.6s
[CV 2/3; 609/648] START gamma=10, learning_rate=0.1, max_depth=10, n_estimators=100, reg_alpha=1, reg_lambda=3
[CV 2/3; 609/648] END gamma=10, learning_rate=0.1, max_depth=10, n_estimators=100, reg_alpha=1, reg_lambda=3;, score=(train=0.715, test=0.693) total time=   1.9s
[CV 3/3; 609/648] START gamma=10, learning_rate=0.1, max_depth=10, n_estimators=100, reg_alpha=1, reg_lambda=3
[CV 3/3; 609/648] END gamma=10, learning_rate=0.1, max_depth=10, n_estimators=100, reg_alpha=1, reg_lambda=3;, score=(train=0.711, test=0.690) total time=   2.2s
[CV 1/3; 610/648] START gamma=10, learning_rate=0.1, max_depth=10, n_estimators=100, reg_alpha=3, reg_lambda=0
[CV 1/3; 610/648] END gamma=10, learning_rate=0.1, max_depth=10, n_estimators=100, reg_alpha=3, reg_lambda=0;, score=(train=0.707, test=0.696) total time=   2.4s
[CV 2/3; 610/648] S

[CV 2/3; 619/648] END gamma=10, learning_rate=0.1, max_depth=10, n_estimators=150, reg_alpha=3, reg_lambda=0;, score=(train=0.710, test=0.692) total time=   2.7s
[CV 3/3; 619/648] START gamma=10, learning_rate=0.1, max_depth=10, n_estimators=150, reg_alpha=3, reg_lambda=0
[CV 3/3; 619/648] END gamma=10, learning_rate=0.1, max_depth=10, n_estimators=150, reg_alpha=3, reg_lambda=0;, score=(train=0.710, test=0.690) total time=   2.0s
[CV 1/3; 620/648] START gamma=10, learning_rate=0.1, max_depth=10, n_estimators=150, reg_alpha=3, reg_lambda=1
[CV 1/3; 620/648] END gamma=10, learning_rate=0.1, max_depth=10, n_estimators=150, reg_alpha=3, reg_lambda=1;, score=(train=0.707, test=0.696) total time=   2.2s
[CV 2/3; 620/648] START gamma=10, learning_rate=0.1, max_depth=10, n_estimators=150, reg_alpha=3, reg_lambda=1
[CV 2/3; 620/648] END gamma=10, learning_rate=0.1, max_depth=10, n_estimators=150, reg_alpha=3, reg_lambda=1;, score=(train=0.711, test=0.692) total time=   2.6s
[CV 3/3; 620/648] S

[CV 3/3; 629/648] END gamma=10, learning_rate=0.1, max_depth=15, n_estimators=50, reg_alpha=3, reg_lambda=1;, score=(train=0.713, test=0.691) total time=   3.2s
[CV 1/3; 630/648] START gamma=10, learning_rate=0.1, max_depth=15, n_estimators=50, reg_alpha=3, reg_lambda=3
[CV 1/3; 630/648] END gamma=10, learning_rate=0.1, max_depth=15, n_estimators=50, reg_alpha=3, reg_lambda=3;, score=(train=0.711, test=0.697) total time=   2.9s
[CV 2/3; 630/648] START gamma=10, learning_rate=0.1, max_depth=15, n_estimators=50, reg_alpha=3, reg_lambda=3
[CV 2/3; 630/648] END gamma=10, learning_rate=0.1, max_depth=15, n_estimators=50, reg_alpha=3, reg_lambda=3;, score=(train=0.713, test=0.693) total time=   3.3s
[CV 3/3; 630/648] START gamma=10, learning_rate=0.1, max_depth=15, n_estimators=50, reg_alpha=3, reg_lambda=3
[CV 3/3; 630/648] END gamma=10, learning_rate=0.1, max_depth=15, n_estimators=50, reg_alpha=3, reg_lambda=3;, score=(train=0.713, test=0.691) total time=   3.1s
[CV 1/3; 631/648] START ga

[CV 1/3; 640/648] END gamma=10, learning_rate=0.1, max_depth=15, n_estimators=150, reg_alpha=0, reg_lambda=0;, score=(train=0.754, test=0.697) total time=   5.3s
[CV 2/3; 640/648] START gamma=10, learning_rate=0.1, max_depth=15, n_estimators=150, reg_alpha=0, reg_lambda=0
[CV 2/3; 640/648] END gamma=10, learning_rate=0.1, max_depth=15, n_estimators=150, reg_alpha=0, reg_lambda=0;, score=(train=0.757, test=0.693) total time=   4.5s
[CV 3/3; 640/648] START gamma=10, learning_rate=0.1, max_depth=15, n_estimators=150, reg_alpha=0, reg_lambda=0
[CV 3/3; 640/648] END gamma=10, learning_rate=0.1, max_depth=15, n_estimators=150, reg_alpha=0, reg_lambda=0;, score=(train=0.755, test=0.692) total time=   4.3s
[CV 1/3; 641/648] START gamma=10, learning_rate=0.1, max_depth=15, n_estimators=150, reg_alpha=0, reg_lambda=1
[CV 1/3; 641/648] END gamma=10, learning_rate=0.1, max_depth=15, n_estimators=150, reg_alpha=0, reg_lambda=1;, score=(train=0.731, test=0.696) total time=   4.5s
[CV 2/3; 641/648] S

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster='gbtree',
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False,
                                     eval_metric='auc', gamma=None, gpu_id=None,
                                     grow_policy='lossguide',
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weig...
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method='hist',
                         

In [76]:
gridsearch_bestModel = clf.best_estimator_

In [77]:
pred_classGS = gridsearch_bestModel.predict(X_test)
pred_probaGS = gridsearch_bestModel.predict_proba(X_test)

In [104]:
print("Accuracy is {0:.4f}".format(accuracy_score(y_test, pred_classGS)))
print("Precision is {0:.4f}".format(precision_score(y_test, pred_classGS)))
print("Recall is {0:.4f}".format(recall_score(y_test, pred_classGS)))
print("AUC score is {0:.4f}".format(roc_auc_score(y_test, pred_probaGS[:,-1])))

Accuracy is 0.6360
Precision is 0.6219
Recall is 0.5419
AUC score is 0.6940


Otrzymany wynik jest nieco lepszy aniżeli w przypadku domyślnego XGBClassifier'a.

## Randomized Search

In [44]:
param_grid = {'gamma': [0,0.1,0.2,0.4,0.8,1.5,3,6,12,25,50,100,200],
              'learning_rate': [0.01, 0.03, 0.06, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7],
              'max_depth': [5,6,7,8,9,10,11,12,13,14],
              'n_estimators': [50,65,80,100,115,130,150],
              'reg_alpha': [0,0.1,0.2,0.4,0.8,1.5,3,6,12,25,50,100,200],
              'reg_lambda': [0,0.1,0.2,0.4,0.8,1.5,3,6,12,25,50,100,200]}

In [49]:
clf_random = RandomizedSearchCV(xgbc0, param_grid, verbose=10, return_train_score=True, scoring = "roc_auc", n_iter=1500)
clf_random.fit(X_train, y_train)

Fitting 5 folds for each of 1500 candidates, totalling 7500 fits
[CV 1/5; 1/1500] START gamma=3.2, learning_rate=0.2, max_depth=7, n_estimators=50, reg_alpha=6.4, reg_lambda=0.2
[CV 1/5; 1/1500] END gamma=3.2, learning_rate=0.2, max_depth=7, n_estimators=50, reg_alpha=6.4, reg_lambda=0.2;, score=(train=0.722, test=0.703) total time=   0.5s
[CV 2/5; 1/1500] START gamma=3.2, learning_rate=0.2, max_depth=7, n_estimators=50, reg_alpha=6.4, reg_lambda=0.2
[CV 2/5; 1/1500] END gamma=3.2, learning_rate=0.2, max_depth=7, n_estimators=50, reg_alpha=6.4, reg_lambda=0.2;, score=(train=0.723, test=0.699) total time=   0.6s
[CV 3/5; 1/1500] START gamma=3.2, learning_rate=0.2, max_depth=7, n_estimators=50, reg_alpha=6.4, reg_lambda=0.2
[CV 3/5; 1/1500] END gamma=3.2, learning_rate=0.2, max_depth=7, n_estimators=50, reg_alpha=6.4, reg_lambda=0.2;, score=(train=0.725, test=0.699) total time=   0.6s
[CV 4/5; 1/1500] START gamma=3.2, learning_rate=0.2, max_depth=7, n_estimators=50, reg_alpha=6.4, reg_la

[CV 5/5; 6/1500] END gamma=12.8, learning_rate=0.7, max_depth=11, n_estimators=50, reg_alpha=0.2, reg_lambda=0.8;, score=(train=0.707, test=0.686) total time=   1.2s
[CV 1/5; 7/1500] START gamma=1.6, learning_rate=0.7, max_depth=10, n_estimators=65, reg_alpha=0.8, reg_lambda=6.4
[CV 1/5; 7/1500] END gamma=1.6, learning_rate=0.7, max_depth=10, n_estimators=65, reg_alpha=0.8, reg_lambda=6.4;, score=(train=0.919, test=0.662) total time=   1.5s
[CV 2/5; 7/1500] START gamma=1.6, learning_rate=0.7, max_depth=10, n_estimators=65, reg_alpha=0.8, reg_lambda=6.4
[CV 2/5; 7/1500] END gamma=1.6, learning_rate=0.7, max_depth=10, n_estimators=65, reg_alpha=0.8, reg_lambda=6.4;, score=(train=0.914, test=0.658) total time=   1.3s
[CV 3/5; 7/1500] START gamma=1.6, learning_rate=0.7, max_depth=10, n_estimators=65, reg_alpha=0.8, reg_lambda=6.4
[CV 3/5; 7/1500] END gamma=1.6, learning_rate=0.7, max_depth=10, n_estimators=65, reg_alpha=0.8, reg_lambda=6.4;, score=(train=0.920, test=0.652) total time=   1.

[CV 4/5; 12/1500] END gamma=51.2, learning_rate=0.1, max_depth=14, n_estimators=130, reg_alpha=0.1, reg_lambda=3.2;, score=(train=0.684, test=0.676) total time=   6.5s
[CV 5/5; 12/1500] START gamma=51.2, learning_rate=0.1, max_depth=14, n_estimators=130, reg_alpha=0.1, reg_lambda=3.2
[CV 5/5; 12/1500] END gamma=51.2, learning_rate=0.1, max_depth=14, n_estimators=130, reg_alpha=0.1, reg_lambda=3.2;, score=(train=0.684, test=0.679) total time=   8.2s
[CV 1/5; 13/1500] START gamma=200, learning_rate=0.7, max_depth=8, n_estimators=130, reg_alpha=25.6, reg_lambda=0.1
[CV 1/5; 13/1500] END gamma=200, learning_rate=0.7, max_depth=8, n_estimators=130, reg_alpha=25.6, reg_lambda=0.1;, score=(train=0.657, test=0.659) total time=   2.0s
[CV 2/5; 13/1500] START gamma=200, learning_rate=0.7, max_depth=8, n_estimators=130, reg_alpha=25.6, reg_lambda=0.1
[CV 2/5; 13/1500] END gamma=200, learning_rate=0.7, max_depth=8, n_estimators=130, reg_alpha=25.6, reg_lambda=0.1;, score=(train=0.657, test=0.655) 

[CV 3/5; 18/1500] END gamma=25.6, learning_rate=0.01, max_depth=8, n_estimators=130, reg_alpha=25.6, reg_lambda=0;, score=(train=0.676, test=0.675) total time=   1.6s
[CV 4/5; 18/1500] START gamma=25.6, learning_rate=0.01, max_depth=8, n_estimators=130, reg_alpha=25.6, reg_lambda=0
[CV 4/5; 18/1500] END gamma=25.6, learning_rate=0.01, max_depth=8, n_estimators=130, reg_alpha=25.6, reg_lambda=0;, score=(train=0.676, test=0.669) total time=   1.6s
[CV 5/5; 18/1500] START gamma=25.6, learning_rate=0.01, max_depth=8, n_estimators=130, reg_alpha=25.6, reg_lambda=0
[CV 5/5; 18/1500] END gamma=25.6, learning_rate=0.01, max_depth=8, n_estimators=130, reg_alpha=25.6, reg_lambda=0;, score=(train=0.677, test=0.671) total time=   1.6s
[CV 1/5; 19/1500] START gamma=0.4, learning_rate=0.1, max_depth=9, n_estimators=150, reg_alpha=0.1, reg_lambda=0
[CV 1/5; 19/1500] END gamma=0.4, learning_rate=0.1, max_depth=9, n_estimators=150, reg_alpha=0.1, reg_lambda=0;, score=(train=0.838, test=0.699) total tim

[CV 2/5; 24/1500] END gamma=6.4, learning_rate=0.7, max_depth=7, n_estimators=80, reg_alpha=0.1, reg_lambda=0.8;, score=(train=0.745, test=0.685) total time=   0.8s
[CV 3/5; 24/1500] START gamma=6.4, learning_rate=0.7, max_depth=7, n_estimators=80, reg_alpha=0.1, reg_lambda=0.8
[CV 3/5; 24/1500] END gamma=6.4, learning_rate=0.7, max_depth=7, n_estimators=80, reg_alpha=0.1, reg_lambda=0.8;, score=(train=0.739, test=0.684) total time=   0.6s
[CV 4/5; 24/1500] START gamma=6.4, learning_rate=0.7, max_depth=7, n_estimators=80, reg_alpha=0.1, reg_lambda=0.8
[CV 4/5; 24/1500] END gamma=6.4, learning_rate=0.7, max_depth=7, n_estimators=80, reg_alpha=0.1, reg_lambda=0.8;, score=(train=0.766, test=0.682) total time=   0.7s
[CV 5/5; 24/1500] START gamma=6.4, learning_rate=0.7, max_depth=7, n_estimators=80, reg_alpha=0.1, reg_lambda=0.8
[CV 5/5; 24/1500] END gamma=6.4, learning_rate=0.7, max_depth=7, n_estimators=80, reg_alpha=0.1, reg_lambda=0.8;, score=(train=0.754, test=0.684) total time=   0.8

...

[CV 5/5; 1479/1500] END gamma=51.2, learning_rate=0.03, max_depth=9, n_estimators=80, reg_alpha=0.1, reg_lambda=0.2;, score=(train=0.682, test=0.676) total time=   1.8s
[CV 1/5; 1480/1500] START gamma=12.8, learning_rate=0.6, max_depth=7, n_estimators=150, reg_alpha=25.6, reg_lambda=0.4
[CV 1/5; 1480/1500] END gamma=12.8, learning_rate=0.6, max_depth=7, n_estimators=150, reg_alpha=25.6, reg_lambda=0.4;, score=(train=0.691, test=0.695) total time=   1.7s
[CV 2/5; 1480/1500] START gamma=12.8, learning_rate=0.6, max_depth=7, n_estimators=150, reg_alpha=25.6, reg_lambda=0.4
[CV 2/5; 1480/1500] END gamma=12.8, learning_rate=0.6, max_depth=7, n_estimators=150, reg_alpha=25.6, reg_lambda=0.4;, score=(train=0.694, test=0.688) total time=   1.6s
[CV 3/5; 1480/1500] START gamma=12.8, learning_rate=0.6, max_depth=7, n_estimators=150, reg_alpha=25.6, reg_lambda=0.4
[CV 3/5; 1480/1500] END gamma=12.8, learning_rate=0.6, max_depth=7, n_estimators=150, reg_alpha=25.6, reg_lambda=0.4;, score=(train=0.

[CV 4/5; 1485/1500] END gamma=0, learning_rate=0.15, max_depth=12, n_estimators=115, reg_alpha=3.2, reg_lambda=0;, score=(train=0.951, test=0.682) total time=   4.0s
[CV 5/5; 1485/1500] START gamma=0, learning_rate=0.15, max_depth=12, n_estimators=115, reg_alpha=3.2, reg_lambda=0
[CV 5/5; 1485/1500] END gamma=0, learning_rate=0.15, max_depth=12, n_estimators=115, reg_alpha=3.2, reg_lambda=0;, score=(train=0.949, test=0.686) total time=   4.0s
[CV 1/5; 1486/1500] START gamma=3.2, learning_rate=0.300000012, max_depth=8, n_estimators=130, reg_alpha=0.1, reg_lambda=12.8
[CV 1/5; 1486/1500] END gamma=3.2, learning_rate=0.300000012, max_depth=8, n_estimators=130, reg_alpha=0.1, reg_lambda=12.8;, score=(train=0.757, test=0.703) total time=   1.4s
[CV 2/5; 1486/1500] START gamma=3.2, learning_rate=0.300000012, max_depth=8, n_estimators=130, reg_alpha=0.1, reg_lambda=12.8
[CV 2/5; 1486/1500] END gamma=3.2, learning_rate=0.300000012, max_depth=8, n_estimators=130, reg_alpha=0.1, reg_lambda=12.8;

[CV 3/5; 1491/1500] END gamma=25.6, learning_rate=0.6, max_depth=13, n_estimators=150, reg_alpha=1.6, reg_lambda=0.1;, score=(train=0.693, test=0.689) total time=   6.8s
[CV 4/5; 1491/1500] START gamma=25.6, learning_rate=0.6, max_depth=13, n_estimators=150, reg_alpha=1.6, reg_lambda=0.1
[CV 4/5; 1491/1500] END gamma=25.6, learning_rate=0.6, max_depth=13, n_estimators=150, reg_alpha=1.6, reg_lambda=0.1;, score=(train=0.693, test=0.683) total time=   9.3s
[CV 5/5; 1491/1500] START gamma=25.6, learning_rate=0.6, max_depth=13, n_estimators=150, reg_alpha=1.6, reg_lambda=0.1
[CV 5/5; 1491/1500] END gamma=25.6, learning_rate=0.6, max_depth=13, n_estimators=150, reg_alpha=1.6, reg_lambda=0.1;, score=(train=0.694, test=0.684) total time=   5.8s
[CV 1/5; 1492/1500] START gamma=3.2, learning_rate=0.2, max_depth=7, n_estimators=50, reg_alpha=0.4, reg_lambda=6.4
[CV 1/5; 1492/1500] END gamma=3.2, learning_rate=0.2, max_depth=7, n_estimators=50, reg_alpha=0.4, reg_lambda=6.4;, score=(train=0.728, 

[CV 2/5; 1497/1500] END gamma=25.6, learning_rate=0.03, max_depth=6, n_estimators=80, reg_alpha=0, reg_lambda=0.8;, score=(train=0.687, test=0.683) total time=   1.1s
[CV 3/5; 1497/1500] START gamma=25.6, learning_rate=0.03, max_depth=6, n_estimators=80, reg_alpha=0, reg_lambda=0.8
[CV 3/5; 1497/1500] END gamma=25.6, learning_rate=0.03, max_depth=6, n_estimators=80, reg_alpha=0, reg_lambda=0.8;, score=(train=0.687, test=0.685) total time=   0.8s
[CV 4/5; 1497/1500] START gamma=25.6, learning_rate=0.03, max_depth=6, n_estimators=80, reg_alpha=0, reg_lambda=0.8
[CV 4/5; 1497/1500] END gamma=25.6, learning_rate=0.03, max_depth=6, n_estimators=80, reg_alpha=0, reg_lambda=0.8;, score=(train=0.688, test=0.679) total time=   1.0s
[CV 5/5; 1497/1500] START gamma=25.6, learning_rate=0.03, max_depth=6, n_estimators=80, reg_alpha=0, reg_lambda=0.8
[CV 5/5; 1497/1500] END gamma=25.6, learning_rate=0.03, max_depth=6, n_estimators=80, reg_alpha=0, reg_lambda=0.8;, score=(train=0.687, test=0.680) tot

RandomizedSearchCV(estimator=XGBClassifier(base_score=None, booster='gbtree',
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           enable_categorical=False,
                                           eval_metric='auc', gamma=None,
                                           gpu_id=None, grow_policy='lossguide',
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_wei...
                                        'learning_rate': [0.01, 0.03, 0.06, 0.1,
                                                          0.15, 0.2, 0.25,
  

In [56]:
randomSearch_bestModel = clf_random.best_estimator_

In [57]:
pred_classRS = randomSearch_bestModel.predict(X_test)
pred_probaRS = randomSearch_bestModel.predict_proba(X_test)

In [103]:
print("Accuracy is {0:.4f}".format(accuracy_score(y_test, pred_classRS)))
print("Precision is {0:.4f}".format(precision_score(y_test, pred_classRS)))
print("Recall is {0:.4f}".format(recall_score(y_test, pred_classRS)))
print("AUC score is {0:.4f}".format(roc_auc_score(y_test, pred_probaRS[:,-1])))

Accuracy is 0.6415
Precision is 0.6246
Recall is 0.5618
AUC score is 0.6963


Wynik jest lepszy od poprzednich dwóch.

## Bayes optimalization

In [14]:
import scipy, skopt

In [15]:
param_grid = {'gamma': skopt.space.space.Real(low = 0.01, high = 200),
              'learning_rate': skopt.space.space.Real(low = 0.01, high = 1),
              'max_depth': skopt.space.space.Integer(5, 20),
              'n_estimators': skopt.space.space.Integer(1, 500),
              'reg_alpha': skopt.space.space.Real(low = 0.01, high = 200),
              'reg_lambda': skopt.space.space.Real(low = 0.01, high = 200)}

In [16]:
from sklearn.model_selection import StratifiedKFold

In [19]:
xgboost_classifier = xgb.XGBClassifier(objective='binary:logistic',
                          booster='gbtree',
                          eval_metric='auc',
                          tree_method='hist',
                          grow_policy='lossguide',
                          use_label_encoder=False)

optimizer = skopt.BayesSearchCV(
            estimator=xgboost_classifier,
            search_spaces=param_grid,
            scoring = 'roc_auc',
            cv=StratifiedKFold(
                n_splits=3,
                shuffle=True
            ),
            n_jobs = -1,
            n_iter = 300,
            verbose = 10
        )

optimizer.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

BayesSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=True),
              estimator=XGBClassifier(base_score=None, booster='gbtree',
                                      colsample_bylevel=None,
                                      colsample_bynode=None,
                                      colsample_bytree=None,
                                      enable_categorical=False,
                                      eval_metric='auc', gamma=None,
                                      gpu_id=None, grow_policy='lossguide',
                                      importance_type=None,
                                      interaction_constraints=None,
                                      learning_r...
                             'learning_rate': Real(low=0.01, high=1, prior='uniform', transform='normalize'),
                             'max_depth': Integer(low=5, high=20, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=1

In [20]:
bayes_model = optimizer.best_estimator_

In [21]:
pred_classB = bayes_model.predict(X_test)
pred_probaB = bayes_model.predict_proba(X_test)

In [24]:
print("Accuracy is {0:.4f}".format(accuracy_score(y_test, pred_classB)))
print("Precision is {0:.4f}".format(precision_score(y_test, pred_classB)))
print("Recall is {0:.4f}".format(recall_score(y_test, pred_classB)))
print("AUC score is {0:.4f}".format(roc_auc_score(y_test, pred_probaB[:,-1])))

Accuracy is 0.6406
Precision is 0.6230
Recall is 0.5629
AUC score is 0.6957


**Jak widać najlepszy model uzykaliśmy dzięki RandomizedSearchCV.**